***
# Тестовое задание:

На основании данных необходимо считать следующий набор метрик:

* Количество показов и уникальных пользователей за день в разрезе по платформам, в том числе по всем платформам суммарно;
* Количество за день уникальных авторов и уникального контента, показанного в ленте;
* Количество сессий, средняя глубина просмотра (по позиции фида) и средняя продолжительность пользовательской сессии в ленте за день.

Напишите алгоритмы / запросы для их расчета.

Если это необходимо – допускается создание промежуточных объектов.

### Описание данных
Данные по показам в ленте. Одна строчка – показ одного «фида» в ленте.

Фидом называем один логический элемент
Описание полей:

* durationMs – время в течение которого пользователь видел фид
* position – позиция в ленте на которой был показан фид
* owners – создатели контента в фиде
* resources – ссылки на ID контента, содержащегося в фиде






In [2]:
#import json
import pandas as pd
import psycopg2

import os
from dotenv import load_dotenv

import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
        
from types import SimpleNamespace        
from sqlalchemy import create_engine  

***
## Quering Data in PostgreSQL

In [6]:
#### Connecting to PostgreSQL Database

# DEFINE THE DATABASE CREDENTIALS
load_dotenv()


user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = int(os.getenv("POSTGRES_PORT"))
database = os.getenv("POSTGRES_DB")

postgres_credentials = SimpleNamespace(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host=os.getenv("POSTGRES_HOST"),
        port=int(os.getenv("POSTGRES_PORT")),
        database=os.getenv("POSTGRES_DB"),
    )

In [7]:
def get_conn(args: SimpleNamespace) -> create_engine:
    return create_engine(
        url=f"postgresql://{args.user}:{args.password}@{args.host}:{args.port}/{args.database}"
    )



In [8]:
engine = get_conn(postgres_credentials)

#### Количество показов и уникальных пользователей за день в разрезе по платформам, в том числе по всем платформам суммарно

In [9]:
query_1 = """


with CTE as 
(
	SELECT 
	a."Date" as date,
	a.platform as platform,
	count(a.index) as views_,
	count(distinct a."userId")  as unique_users

	FROM public.df_1_users a
	group by a."Date", a.platform
	order by 1,2 
)
select 
	date, 
	platform,
	views_,
	sum(views_) over (partition by date) as views_by_day,
	unique_users,
	sum(unique_users) over (partition by date) as unique_users_by_day
from CTE
"""

In [10]:
pd.read_sql_query(sql=query_1,con=engine)

,date,platform,views_,views_by_day,unique_users,unique_users_by_day
0,2019-09-12,APP_ANDROID,18078,31230.0,109,200.0
1,2019-09-12,APP_IOS,2378,31230.0,23,200.0
2,2019-09-12,APP_WINPHONE,247,31230.0,1,200.0
3,2019-09-12,DESKTOP_WEB,9177,31230.0,52,200.0
4,2019-09-12,MOBILE_WEB,1350,31230.0,15,200.0
5,2019-09-13,APP_ANDROID,1793,3088.0,30,47.0
6,2019-09-13,APP_IOS,92,3088.0,4,47.0
7,2019-09-13,DESKTOP_WEB,1175,3088.0,11,47.0
8,2019-09-13,MOBILE_WEB,28,3088.0,2,47.0


#### Количество за день уникальных авторов и уникального контента, показанного в ленте

In [11]:
query_2 = """


with CTE as
(
SELECT t1."Date", t1."_owners", t1."owners_values", t2."resources", t2."resources_id" 
FROM public.df_1_users t1 LEFT JOIN df_2_content t2 
ON t1."owners_values" = t2."owners_values" and t1."timestamp" = t2."timestamp"
)
		select 
			a."Date",
			count(distinct a."owners_values") as unique_owners,
			count(distinct a."resources_id") as unique_content
		from CTE a
			group by a."Date"
            
"""

In [12]:
pd.read_sql_query(sql=query_2,con=engine)

,Date,unique_owners,unique_content
0,2019-09-12,10274,38905
1,2019-09-13,1264,5526


#### Количество сессий, средняя глубина просмотра (по позиции фида) и средняя продолжительность пользовательской сессии в ленте за день.

Глубина просмотра (или посещения) сайта — это среднее число просмотров контента сайта пользователями за одно посещение

In [13]:
query_3 = """

SELECT 
	a."Date",
	a."position",
	count(a.index) as count_session,
	count(a.index)/count( distinct a."userId") as avg_sesion_by_user_sec,
    round(sum(a."durationMs")/count( distinct a."userId")/1000) as avg_sesion_by_user_sec

FROM public.df_1_users a
group by a."Date", a."position"
order by a."Date", a."position"


"""

In [14]:
pd.read_sql_query(sql=query_3,con=engine)

,Date,position,count_session,avg_sesion_by_user_sec,avg_sesion_by_user_sec
0,2019-09-12,1.0,859,4,521.0
1,2019-09-12,2.0,658,4,323.0
2,2019-09-12,3.0,580,3,116.0
3,2019-09-12,4.0,553,3,331.0
4,2019-09-12,5.0,509,3,43.0
...,...,...,...,...,...
1037,2019-09-13,428.0,1,1,NaN
1038,2019-09-13,429.0,1,1,NaN
1039,2019-09-13,430.0,1,1,NaN
1040,2019-09-13,431.0,1,1,NaN
